# Create an org.json, convert that to the new json

In [19]:
import os
import json
import numpy as np
import glob
from skimage.io import imread, imsave
from collections import Counter

In [21]:
data_dir = os.path.expanduser('~/google_drive/scratch/starfish/linnarsson/TIFF/series_1/')

In [24]:
metadata = {
    "num_hybs": 1,
    "num_chs": 2,
    "shape": [2048, 2048],
    "is_volume": True,
    "format": "TIFF"
}

In [25]:
fluor_to_file_mapping = {
    "c001": 0,
    "c002": 1,
}

In [29]:
data = []
for f in glob.glob(os.path.join(data_dir, '*.tif')):
    if 'c003' in f: 
        continue  # DAPI
    _, _, suffix = f.partition('100X_')
    file_ = os.path.basename(f)
    z, ch = suffix.strip('.tif').split('_')
    data.append({
        "hyb": 0,
        "ch": fluor_to_file_mapping[ch],
        "file": file_,
        "z": int(z[1:]) - 1  # one based. I guess at some point I should just write a way to do this from the new spec. 
    })

In [30]:
# grab the dapi images and max_project them, then define that result

# get all z-images
dapi_data = []
for f in glob.glob(os.path.join(data_dir, '*.tif')):
    if 'c003' in f: 
        dapi_data.append(f)  # DAPI
    _, _, suffix = f.partition('100X_')
    file_ = os.path.basename(f)
    z, ch = suffix.strip('.tif').split('_')

# concatenate them over z
np_data = [imread(f) for f in dapi_data]
x, y = np_data[0].shape
z = len(np_data)
volume = np.zeros((x, y, z), dtype=np_data[0].dtype)
for i, m in enumerate(np_data): 
    volume[:, :, i] = m

# take max over z
maxproj = np.max(volume, 2)

# write result
imsave(os.path.join(data_dir, 'c003_max_projection.tif'), maxproj)

In [33]:
# define the aux section
# fake this by just including one of them
aux = [
    {
        "type": "dapi",
        "file": os.path.join(data_dir, 'c003_max_projection.tif'),
        "format": "TIFF"
    }
]

In [35]:
with open(os.path.join(data_dir, 'org.json'), 'w') as f:
    json.dump(dict(data=data, aux=aux, metadata=metadata), f)

# Load the new spec into Starfish

In [6]:
from starfish.io import Stack

In [36]:
s = Stack()
s.read(os.path.join(data_dir, 'experiment.json'))